![car](car.jpg)

Insurance companies invest a lot of [time and money](https://www.accenture.com/_acnmedia/pdf-84/accenture-machine-leaning-insurance.pdf) into optimizing their pricing and accurately estimating the likelihood that customers will make a claim. In many countries insurance it is a legal requirement to have car insurance in order to drive a vehicle on public roads, so the market is very large!

Knowing all of this, On the Road car insurance have requested your services in building a model to predict whether a customer will make a claim on their insurance during the policy period. As they have very little expertise and infrastructure for deploying and monitoring machine learning models, they've asked you to identify the single feature that results in the best performing model, as measured by accuracy, so they can start with a simple model in production.

They have supplied you with their customer data as a csv file called `car_insurance.csv`, along with a table detailing the column names and descriptions below.



## The dataset

| Column | Description |
|--------|-------------|
| `id` | Unique client identifier |
| `age` | Client's age: <br> <ul><li>`0`: 16-15</li><li>`1`: 26-39</li><li>`2`: 40-64</li><li>`3`: 65+</li></ul> |
| `gender` | Client's gender: <br> <ul><li>`0`: Female</li><li>`1`: Male</li></ul> |
| `driving_experience` | Years the client has been driving: <br> <ul><li>`0`: 0-9</li><li>`1`: 10-19</li><li>`2`: 20-29</li><li>`3`: 30+</li></ul> |
| `education` | Client's level of education: <br> <ul><li>`0`: No education</li><li>`1`: High school</li><li>`2`: University</li></ul> |
| `income` | Client's income level: <br> <ul><li>`0`: Poverty</li><li>`1`: Working class</li><li>`2`: Middle class</li><li>`3`: Upper class</li></ul> |
| `credit_score` | Client's credit score (between zero and one) |
| `vehicle_ownership` | Client's vehicle ownership status: <br><ul><li>`0`: Does not own their vehilce (paying off finance)</li><li>`1`: Owns their vehicle</li></ul> |
| `vehcile_year` | Year of vehicle registration: <br><ul><li>`0`: Before 2015</li><li>`1`: 2015 or later</li></ul> |
| `married` | Client's marital status: <br><ul><li>`0`: Not married</li><li>`1`: Married</li></ul> |
| `children` | Client's number of children |
| `postal_code` | Client's postal code | 
| `annual_mileage` | Number of miles driven by the client each year |
| `vehicle_type` | Type of car: <br> <ul><li>`0`: Sedan</li><li>`1`: Sports car</li></ul> |
| `speeding_violations` | Total number of speeding violations received by the client | 
| `duis` | Number of times the client has been caught driving under the influence of alcohol |
| `past_accidents` | Total number of previous accidents the client has been involved in |
| `outcome` | Whether the client made a claim on their car insurance (response variable): <br><ul><li>`0`: No claim</li><li>`1`: Made a claim</li></ul> |

In [4]:
# Import required modules
import pandas as pd
import numpy as np
from statsmodels.formula.api import logit

# Start coding!
car_df = pd.read_csv('car_insurance.csv')
car_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   10000 non-null  int64  
 1   age                  10000 non-null  int64  
 2   gender               10000 non-null  int64  
 3   driving_experience   10000 non-null  object 
 4   education            10000 non-null  object 
 5   income               10000 non-null  object 
 6   credit_score         9018 non-null   float64
 7   vehicle_ownership    10000 non-null  float64
 8   vehicle_year         10000 non-null  object 
 9   married              10000 non-null  float64
 10  children             10000 non-null  float64
 11  postal_code          10000 non-null  int64  
 12  annual_mileage       9043 non-null   float64
 13  vehicle_type         10000 non-null  object 
 14  speeding_violations  10000 non-null  int64  
 15  duis                 10000 non-null  

In [5]:
car_df['credit_score'] = car_df['credit_score'].fillna(car_df['credit_score'].mean())

car_df['annual_mileage'] = car_df['annual_mileage'].fillna(car_df['annual_mileage'].mean())

In [9]:
car_df = car_df.drop('id', axis=1)

In [11]:
X = car_df.drop('outcome', axis=1)
y = car_df['outcome']

categorical_cols = X.select_dtypes(include=['object', 'category']).columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

scaler = StandardScaler()
ohe = OneHotEncoder(handle_unknown='ignore')

# Scale numerical columns
X_train_num = scaler.fit_transform(X_train[numerical_cols])
X_test_num = scaler.transform(X_test[numerical_cols])

# One-hot encode categorical columns
X_train_cat = ohe.fit_transform(X_train[categorical_cols]).toarray()
X_test_cat = ohe.transform(X_test[categorical_cols]).toarray()



In [22]:
# Convert to DataFrame
X_train_num_df = pd.DataFrame(X_train_num, columns=numerical_cols, index=X_train.index)
X_train_cat_df = pd.DataFrame(X_train_cat, columns=ohe.get_feature_names(categorical_cols), index=X_train.index)
X_test_num_df = pd.DataFrame(X_test_num, columns=numerical_cols, index=X_test.index)
X_test_cat_df = pd.DataFrame(X_test_cat, columns=ohe.get_feature_names(categorical_cols), index=X_test.index)

In [23]:
# Combine numerical and categorical columns back into a dataframe
X_train_preprocessed = pd.DataFrame(X_train_num, columns=numerical_cols).join(pd.DataFrame(X_train_cat, columns=ohe.get_feature_names(categorical_cols)))
X_test_preprocessed = pd.DataFrame(X_test_num, columns=numerical_cols).join(pd.DataFrame(X_test_cat, columns=ohe.get_feature_names(categorical_cols)))


In [32]:
model = LogisticRegression()
model.fit(X_train_preprocessed, y_train)

y_pred = model.predict(X_test_preprocessed)


In [34]:
from sklearn.metrics import accuracy_score, confusion_matrix

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)

print(confusion_matrix(y_pred, y_test))
print(f"Accuracy of the model: {accuracy:.2f}")

[[1213  168]
 [ 160  459]]
Accuracy of the model: 0.84


In [43]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Define models and their hyperparameters
models = {
    'LogisticRegression': {
        'model': LogisticRegression(),
        'params': {
            'C': np.logspace(-4, 4, 20),
            'penalty': ['l1', 'l2']
        }
    },
    'RandomForest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [10, 50, 100, 200],
            'max_features': ['auto', 'sqrt', 'log2'],
            'max_depth': [None, 5, 10, 15, 20]
        }
    }
}

# List for storing best models
best_models = []

# Hyperparameter tuning using RandomizedSearchCV
for model_name, mp in models.items():
    random_search = RandomizedSearchCV(mp['model'], mp['params'], n_iter=10, cv=5, scoring='accuracy', random_state=42)
    random_search.fit(X_train_preprocessed, y_train)
    best_models.append({
        'model': model_name,
        'best_score': random_search.best_score_,
        'best_params': random_search.best_params_
    })

# Find the best model
best_model = max(best_models, key=lambda x: x['best_score'])

print(f"Best Model: {best_model['model']}")
print(f"Best Score: {best_model['best_score']:.2f}")
print(f"Best Hyperparameters: {best_model['best_params']}")


Best Model: LogisticRegression
Best Score: 0.85
Best Hyperparameters: {'penalty': 'l2', 'C': 3792.690190732246}
